## Web scrapping for lyrics

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LinearRegression
from sklearn import linear_model
from sklearn.preprocessing import PolynomialFeatures
import sklearn.metrics as sm

import time

#Genius
from lyricsgenius import Genius
import lyricsgenius

genius_token = "4SoLPv5UHyFZyAe5T-PhxS-bRCRrdNIgMNpeqjjIaWvnRbUmTpoGxd9knxBtlQyg" # Genius credentials
LyricsGenius = lyricsgenius.Genius(genius_token)


### Importing data set

In [3]:
#data = pd.read_csv('../../data/music-subset.csv')
data = pd.read_csv('../data/cleanedDataSetWithFeatures.csv')

### Preparing data set.

In [4]:
data.shape

(104544, 19)

In [5]:
data.head()

,Unnamed: 0,title,rank,date,artist,url,region,streams,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
0,0,Chantaje (feat. Maluma),1,2017-01-01,Shakira,https://open.spotify.com/track/6mICuAdrwEjh6Y6...,Argentina,253019.0,0.852,0.773,8.0,-2.921,0.0776,0.187,0.000030,0.1590,0.9070,102.034,195840.0
1,1,Despacito (Featuring Daddy Yankee),1,2017-04-07,Luis Fonsi,https://open.spotify.com/track/4aWmUDTfIPGksMN...,Dominican Republic,11681.0,0.660,0.786,2.0,-4.757,0.1700,0.209,0.000000,0.1120,0.8460,177.833,228200.0
2,2,ภาพจำ,1,2018-04-12,Pop Pongkool,https://open.spotify.com/track/4YRpjJaJpiI36HM...,Thailand,15477.0,0.557,0.585,5.0,-7.559,0.0470,0.426,0.000001,0.1440,0.4640,81.922,274000.0
3,3,Te Boté - Remix,1,2018-04-27,"Nio Garcia, Casper Magico, Bad Bunny, Darell, ...",https://open.spotify.com/track/3V8UKqhEK5zBkBb...,Honduras,12869.0,0.903,0.675,11.0,-3.445,0.2140,0.542,0.000013,0.0595,0.4420,96.507,417920.0
4,4,Binibini,1,2021-04-16,Zack Tabudlo,https://open.spotify.com/track/2X5AFygz5SDYlXa...,Philippines,401678.0,0.642,0.374,5.0,-10.606,0.0320,0.423,0.000002,0.4350,0.0979,129.863,221538.0


In [6]:
data.columns

Index(['Unnamed: 0', 'title', 'rank', 'date', 'artist', 'url', 'region',
       'streams', 'danceability', 'energy', 'key', 'loudness', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'duration_ms'],
      dtype='object')

In [7]:
data.drop('Unnamed: 0', inplace=True, axis=1)
data.drop('Unnamed: 0.1', inplace=True, axis=1)

KeyError: "['Unnamed: 0.1'] not found in axis"

In [ ]:
data.isnull().sum()

In [8]:
data = data.dropna()

***
### Creating a subset

In [9]:
# Set the amount of rows in sub set.
reduced_data = data.iloc[0:2000]

In [10]:
# Check the shape of the dataset.
reduced_data.shape

(2000, 18)

In [11]:
# Find link for accessing data for each song.
reduced_data['url'][0]

'https://open.spotify.com/track/6mICuAdrwEjh6Y6lroV2Kg'

***
### Adding lyrics to the list

In [12]:
LyricsGenius.remove_section_headers = True
LyricsGenius.verbose = False

def combineSongAndLyric(dataset):
    # Start timer.
    starttimer = time.perf_counter()
 
    errors = 0

    lyrics = []
    index = []
    
    for i, row in dataset.iterrows():
        try:
            #Track name
            track_name = row['title']
            #Artist name
            artist_name = row['artist']

            removeFeaturedArtist = track_name.find(" (") 
            removeMultipleArtist = artist_name.find(", ")

            if (removeFeaturedArtist > 0): track_name = track_name[:removeFeaturedArtist]
            if (removeMultipleArtist > 0): artist_name = artist_name[:removeMultipleArtist]    

            #Songs Lyrics
            try:
                song_lyric = LyricsGenius.search_song(track_name, artist_name).lyrics
                dataset.at[i,'lyrics'] = song_lyric
            except:
                errors = errors+1
                dataset.at[i,'lyrics'] = 'Coming Soon...'
                pass

            print(str(int(((i+1)/dataset.shape[0])*100)) + '% done - ' + str(i+1) + ' of ' + str(dataset.shape[0]) + ' songs iterated - Errors found: ' + str(errors), end='\r')
            
        except KeyboardInterrupt:
            print('Interrupted')
            try:
                sys.exit(0)
            except SystemExit:
                os._exit(0)

In [ ]:
combineSongAndLyric(reduced_data)

C:\Users\patr5\anaconda3\lib\site-packages\pandas\core\indexing.py:1684: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
C:\Users\patr5\anaconda3\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [ ]:
#Saving found data to csv file
from pathlib import Path  
path = Path('../../data/2000songWithLyrics.csv')  
path.parent.mkdir(parents=True, exist_ok=True)  
reduced_data.to_csv(path)